匯入所需工具 BeautifulSoup, requests, pymysql, pandas
設定sql連線

In [2]:
import pymysql.cursors
from dotenv import load_dotenv, find_dotenv
import os
load_dotenv(find_dotenv())


conn = pymysql.connect(host=os.getenv("MYSQL_DB_HOST"),
                       user=os.getenv("MYSQL_USER"),
                       password=os.getenv("MYSQL_PASSWORD"),
                       database=os.getenv("MYSQL_DB_NAME"),
                       charset='utf8',
                       cursorclass=pymysql.cursors.DictCursor)
cur = conn.cursor()
sql='''CREATE TABLE tabletrail(`步道名稱` char not null,`所在地` char,`網址` char,`步道難度` char,`步道長度` char,`所需時間` int,`熱門度` int,`到過人數` int,`點閱數` int)'''
cur.execute(sql)
conn.commit()
conn.close()

建立所需list
步道名稱,所在地,網址,步道難度,步道長度,所需時間,熱門度,到過人數,點閱數

In [4]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import sqlite3
import pymysql


item_list=[]
area_list=[]
link_list=[]
piclink_list=[]

info_list=[]
diff_list=[]
length_list=[]
time_list=[]


like_list=[]
pop_list=[]
gone_list=[]
view_list=[]

設定爬蟲頁面 資料來源：健行筆記步道資訊頁面 共89頁
取出資料後存入建立的list
執行sql指令 將list匯入sql database

In [ ]:
for page in range(1,86):#爬所有頁面的資訊 共86頁
    res=requests.get("https://hiking.biji.co/index.php?q=trail&page="+ str(page))#第一頁的網址後續用str加頁數
    soup=BeautifulSoup(res.text, "html.parser")
    items = soup.select('h2',class_="text-xl truncate")#取出步道名稱
    items2 = soup.find_all('div',class_="text-base text-gray-600 flex-1 truncate")#取出步道所在位置
    items3 = soup.find_all('ul',class_="flex items-center space-x-2.5")#取出步道其他資訊
    items4 = soup.find_all('li',class_="flex items-center space-x-1.5")#取出步道反饋資訊
    items5 = soup.find_all('a',class_="w-56")#取出步道照片網址

    for i in items:
        item_list.append(i.select_one("a").getText())#取出步道名稱
             
        link_list.append(i.select_one("a").get("href"))#取出步道網址
               
    for i in items5:    
        piclink_list.append(i.select_one("img").get("data-src"))#取出步道照片網址
   
     
    for i in items2:
        area_list.append(i.getText())#取出步道所在地      
        
    for i in items3:
        info_list.append(i.getText())#取出步道資訊三筆合併成一個string需分割    
        
    for i in items4:
        like_list.append(i.getText())#取出步道反饋資訊三筆合併成一個list   
        
pop_list=like_list[0::3]#取出熱門度
gone_list=like_list[1::3]#取出到過人數
view_list=like_list[2::3]#取出點閱數

for i in info_list:#步道資訊整理 爬回的資料需分割
    info=i.split()
    diff_list.append(info[1])#取出步道難度
    length_list.append(info[2])#取出步道長度
    
for i in info_list:#步道資訊整理 爬回的資料需分割
    info=i.split("\n")
    time_list.append(info[3])#取出步道所需時間   
  

for i in range(len(item_list)):# 資料匯入mysql 執行SQL指令
    sql = '''INSERT INTO tabletrail (步道名稱,所在地,照片網址,連結網址,步道難度,步道長度,所需時間,熱門度,到過人數,點閱數) 
              VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s) '''
    cur.execute(sql,(item_list[i],area_list[i],piclink_list[i],link_list[i]
                     ,diff_list[i],length_list[i],time_list[i]
                     ,pop_list[i],gone_list[i],view_list[i]))   
    conn.commit() 